In [ ]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms

# Data loading and splitting
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for CIFAR-100
])

full_dataset = CIFAR100(root="./data", train=True, download=True, transform=transform)

# Split the dataset into training, validation, and testing sets
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 169001437/169001437 [00:02<00:00, 77143703.26it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


# First Run
Note: Forgot to save trained model as "mobilenet_v3_large.pth" instead of "resnet_model.pth"

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
# from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 100  # CIFAR-100 has 100 classes
lr = 0.003 #1e-4
patch_size = 4 #8
num_layers = 12
num_heads = 16 #12
hidden_dim = 512 #768
mlp_dim = 3072
image_size = 32
# max_len = 100 # All sequences must be less than 1000 including class token
# channels = 3

num_epochs = 30

# Model, loss, and optimizer
# weights = ResNet50_Weights.DEFAULT
weights = MobileNet_V3_Large_Weights.DEFAULT

model = mobilenet_v3_large(weights=weights).to(device)
# model = resnet50(weights=weights).to(device)
#model = resnet50(num_classes=num_classes, image_size=image_size, patch_size=patch_size, num_layers=num_layers, num_heads=num_heads, hidden_dim=hidden_dim, mlp_dim=mlp_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Optional: learning rate scheduler

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Optional: Adjust learning rate
    scheduler.step()

    # Print average loss per epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    total_val_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()

    val_accuracy = correct / total
    average_val_loss = total_val_loss / len(val_loader)
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%, , Validation Loss: {average_val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "resnet_model.pth")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 102MB/s]
Epoch 1/30: 100%|██████████| 137/137 [00:14<00:00,  9.36it/s]


Epoch 1/30, Loss: 3.7098


Validation: 100%|██████████| 40/40 [00:02<00:00, 13.65it/s]


Validation Accuracy: 21.77%, , Validation Loss: 3.3256


Epoch 2/30: 100%|██████████| 137/137 [00:14<00:00,  9.73it/s]


...

Validation Accuracy: 52.47%, , Validation Loss: 3.4960


Epoch 30/30: 100%|██████████| 137/137 [00:14<00:00,  9.78it/s]


Epoch 30/30, Loss: 0.0233


Validation: 100%|██████████| 40/40 [00:02<00:00, 13.65it/s]


Validation Accuracy: 52.60%, , Validation Loss: 3.5170


In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Evaluating: 100%|██████████| 20/20 [00:01<00:00, 12.36it/s]

Test Accuracy: 53.60%


# Second Run
Notes:
1.   Forgot to save trained model as "mobilenet_v3_large.pth" instead of "resnet_model.pth"
2.   Changed image size from 32 to 224

In [ ]:
# new run
import torch
import torch.nn as nn
from torchvision import models, transforms
# from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 100  # CIFAR-100 has 100 classes
lr = 0.003 #1e-4
patch_size = 4 #8
num_layers = 12
num_heads = 16 #12
hidden_dim = 512 #768
mlp_dim = 3072
image_size = 224
# max_len = 100 # All sequences must be less than 1000 including class token
# channels = 3

num_epochs = 30

# Model, loss, and optimizer
# weights = ResNet50_Weights.DEFAULT
weights = MobileNet_V3_Large_Weights.DEFAULT

model = mobilenet_v3_large(weights=weights).to(device)
# model = resnet50(weights=weights).to(device)
#model = resnet50(num_classes=num_classes, image_size=image_size, patch_size=patch_size, num_layers=num_layers, num_heads=num_heads, hidden_dim=hidden_dim, mlp_dim=mlp_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Optional: learning rate scheduler

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Optional: Adjust learning rate
    scheduler.step()

    # Print average loss per epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    total_val_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()

    val_accuracy = correct / total
    average_val_loss = total_val_loss / len(val_loader)
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%, , Validation Loss: {average_val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "resnet_model.pth")

Epoch 1/30: 100%|██████████| 137/137 [00:13<00:00, 10.02it/s]


Epoch 1/30, Loss: 3.5275


Validation: 100%|██████████| 40/40 [00:02<00:00, 13.47it/s]


Validation Accuracy: 21.97%, , Validation Loss: 3.6388


Epoch 2/30: 100%|██████████| 137/137 [00:13<00:00, 10.10it/s]


...

Validation Accuracy: 53.01%, , Validation Loss: 3.4938


Epoch 30/30: 100%|██████████| 137/137 [00:13<00:00,  9.97it/s]


Epoch 30/30, Loss: 0.0193


Validation: 100%|██████████| 40/40 [00:03<00:00, 10.01it/s]


Validation Accuracy: 53.20%, , Validation Loss: 3.5055


In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Evaluating: 100%|██████████| 20/20 [00:02<00:00,  7.93it/s]

Test Accuracy: 52.56%


# Third Run
Notes:
1.   Changed image size back to 32
2.   Changed learning rate from 0.003 to 0.001

In [ ]:
# new run
import torch
import torch.nn as nn
from torchvision import models, transforms
# from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 100  # CIFAR-100 has 100 classes
lr = 0.001 #1e-4
patch_size = 4 #8
num_layers = 12
num_heads = 16 #12
hidden_dim = 512 #768
mlp_dim = 3072
image_size = 32
# max_len = 100 # All sequences must be less than 1000 including class token
# channels = 3

num_epochs = 30

# Model, loss, and optimizer
# weights = ResNet50_Weights.DEFAULT
weights = MobileNet_V3_Large_Weights.DEFAULT

model = mobilenet_v3_large(weights=weights).to(device)
# model = resnet50(weights=weights).to(device)
#model = resnet50(num_classes=num_classes, image_size=image_size, patch_size=patch_size, num_layers=num_layers, num_heads=num_heads, hidden_dim=hidden_dim, mlp_dim=mlp_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Optional: learning rate scheduler

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Optional: Adjust learning rate
    scheduler.step()

    # Print average loss per epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    total_val_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()

    val_accuracy = correct / total
    average_val_loss = total_val_loss / len(val_loader)
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%, , Validation Loss: {average_val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "mobilenet_v3_large.pth")

Epoch 1/30: 100%|██████████| 137/137 [00:14<00:00,  9.66it/s]


Epoch 1/30, Loss: 4.1013


Validation: 100%|██████████| 40/40 [00:03<00:00, 11.01it/s]


Validation Accuracy: 22.97%, , Validation Loss: 3.1913


Epoch 2/30: 100%|██████████| 137/137 [00:15<00:00,  9.12it/s]


...

Validation Accuracy: 49.16%, , Validation Loss: 3.4122


Epoch 30/30: 100%|██████████| 137/137 [00:14<00:00,  9.37it/s]


Epoch 30/30, Loss: 0.0288


Validation: 100%|██████████| 40/40 [00:03<00:00, 12.26it/s]


Validation Accuracy: 49.26%, , Validation Loss: 3.4240


In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Evaluating: 100%|██████████| 20/20 [00:01<00:00, 12.25it/s]

Test Accuracy: 48.68%
